In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
## Implementation of DCGAN - as described by Radford et al. 2015
#1. Importing required packages
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop


#2. Constructing DCGAN class that contains the model for the discriminator and generator
class MNISTDCGAN(object):
    def __init__(self):
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    #3. Setting up discriminator model
    def discriminator(self):
        if self.D:
            return self.D
        self.D = Sequential()
        discriminator_filter_size = 64
        

        dropout = 0.3
        #Documentation on the usage of convolutional layers can be found here https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
        self.D.add(Conv2D(discriminator_filter_size, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(discriminator_filter_size*2, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(discriminator_filter_size*4, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(discriminator_filter_size*8, 5, strides=1, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        # Out: 1-dim probability
        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        print("Discrimnator Summary")
        self.D.summary()
        return self.D

    #4. Setting up generator model
    def generator(self):
        if self.G:
            return self.G
        self.G = Sequential()
        dropout = 0.4
        generator_filter_size = 256
        input_dimension = 7
        self.G.add(Dense(input_dimension*input_dimension*generator_filter_size, input_dim=64))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(Reshape((input_dimension, input_dimension, generator_filter_size)))
        self.G.add(Dropout(dropout))
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(generator_filter_size/2), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(generator_filter_size/4), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(Conv2DTranspose(int(generator_filter_size/8), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(Conv2DTranspose(1, 5, padding='same'))
        self.G.add(Activation('tanh'))
        print("Generator Summary")
        self.G.summary()
        return self.G

    #5. Defining discriminator model
    def discriminator_model(self):
        if self.DM:
            return self.DM
        optimizer = RMSprop(lr=0.0002, decay=6e-8)
        self.DM = Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.DM

    #6. Defining generator model
    def adversarial_model(self):
        if self.AM:
            return self.AM
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.AM

#7. Constructing  class that contains the training algorithm for the models
class TrainDCGAN(object):
    def __init__(self):
        '''
        Usage documentation can be found here: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data
        Since this is a unsupervised learning and we wanted only the training images, we ignored to load other x_test, y_train and y_test.
        '''
        (self.x_train, _), (_, _) = mnist.load_data(path="mnist.npz")
        self.x_train = self.x_train / 127.5 - 1.
        self.x_train = np.expand_dims(self.x_train, axis=3)

        #8. Creating objects for DCGAN and calling the models 
        self.DCGAN = MNISTDCGAN()
        self.discriminator =  self.DCGAN.discriminator_model()
        self.adversarial = self.DCGAN.adversarial_model()
        self.generator = self.DCGAN.generator()
        self.disc_loss = []
        self.gen_loss = []

    #9. Defining the training parameters
    def train(self, epochs, batch_size=256, save_interval=1000):
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[batch_size, 64])
        for i in range(epochs):
            images_train = self.x_train[np.random.randint(0, self.x_train.shape[0], size=batch_size), :, :, :]
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 64])
            images_fake = self.generator.predict(noise)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)
            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 64])
            a_loss = self.adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            self.disc_loss.append(d_loss[0])
            self.gen_loss.append(a_loss[0])
            print(log_mesg)
            if (i+1)%save_interval==0:
              self.generate_numbers(i)


    #10. Generating new numbers at given sample interval
    def generate_numbers(self, epoch):
        input_noise = np.random.normal(0, 1, (1, 64))
        predictions = self.generator.predict(input_noise)
        print(predictions.shape)
        print("Reshaping Output data")
        generated_seq = np.reshape(predictions[0], (28,28))
        print(generated_seq.shape)
        plt.figure(figsize = (5,5))
        plt.imshow(generated_seq,aspect='auto',cmap='gray')
        #Path to be created in Google Drive to avoid failure
        plt.savefig("/content/drive/My Drive/DL/GAN/Results/DCGAN_MNIST/Epoch-%d.png" % epoch)
        plt.show()
        plt.close()


In [0]:
gan = TrainDCGAN()
gan.train(epochs=10000, batch_size=32, save_interval=200)

8872: [D loss: 0.727026, acc: 0.468750]  [A loss: 0.954047, acc: 0.031250]
8873: [D loss: 0.654934, acc: 0.609375]  [A loss: 0.827100, acc: 0.281250]
8874: [D loss: 0.701516, acc: 0.578125]  [A loss: 0.814974, acc: 0.250000]
8875: [D loss: 0.694074, acc: 0.453125]  [A loss: 0.821690, acc: 0.218750]
8876: [D loss: 0.708722, acc: 0.500000]  [A loss: 0.966682, acc: 0.093750]
8877: [D loss: 0.717014, acc: 0.515625]  [A loss: 0.839943, acc: 0.281250]
8878: [D loss: 0.689192, acc: 0.515625]  [A loss: 1.066706, acc: 0.000000]
8879: [D loss: 0.701796, acc: 0.484375]  [A loss: 0.857499, acc: 0.093750]
8880: [D loss: 0.729980, acc: 0.500000]  [A loss: 0.995060, acc: 0.031250]
8881: [D loss: 0.740303, acc: 0.500000]  [A loss: 0.789582, acc: 0.343750]
8882: [D loss: 0.678550, acc: 0.593750]  [A loss: 1.022691, acc: 0.062500]
8883: [D loss: 0.710086, acc: 0.531250]  [A loss: 0.790411, acc: 0.281250]
8884: [D loss: 0.696399, acc: 0.578125]  [A loss: 0.904277, acc: 0.093750]
8885: [D loss: 0.716777, 